In [1]:
import itertools
import pandas as pd
from pandas import DataFrame
import re
import subprocess

In [2]:
train = pd.read_csv('gold_standard_v9.csv')
dev = pd.read_csv('dev_df_v9.csv')
test = pd.read_csv('test_df_v9.csv')

Identifying sentencces in the training sets that have repetition, for development of our gold, silver and bronze standard dataset settings

In [24]:
train.head(100000).to_csv('toy_train_df.csv', index=False)
dev.head(4500).to_csv('toy_dev_df.csv', index=False)
test.head(4500).to_csv('toy_test_df.csv', index=False)

In [20]:
train

,wav_filename,wav_filesize,transcript
0,/media/ks/Elements/cv-corpus-9.0-2022-04-27/sw...,199,Spishi zote zinatokea Afrika tu.
1,/media/ks/Elements/cv-corpus-9.0-2022-04-27/sw...,230,Kwa sasa yeye anaishi katika mji mkuu wa Nairo...
2,/media/ks/Elements/cv-corpus-9.0-2022-04-27/sw...,205,"Mmojawapo, maji yanafikia ziwa Tanganyika."
3,/media/ks/Elements/cv-corpus-9.0-2022-04-27/sw...,211,Pia mara nyingi huingia ndani ya milango ya mi...
4,/media/ks/Elements/cv-corpus-9.0-2022-04-27/sw...,249,Kiebrania kilikuwa lugha ya Israeli ya Kale wa...
...,...,...,...
19856,/media/ks/Elements/cv-corpus-9.0-2022-04-27/sw...,137,Walio na mvuto mkubwa kwa kila mtu na ndege hao
19857,/media/ks/Elements/cv-corpus-9.0-2022-04-27/sw...,151,Ziada kwa Tanzania ya dola milioni mia mbili s...
19858,/media/ks/Elements/cv-corpus-9.0-2022-04-27/sw...,123,pep guardiola ameshindwa kumsainisha messi
19859,/media/ks/Elements/cv-corpus-9.0-2022-04-27/sw...,180,Vivutio hivyo vinafikika kila mwaka na kila ki...


In [21]:
toy_train = pd.read_csv('toy_train_df.csv')
toy_dev = pd.read_csv('toy_dev_df.csv')
toy_test = pd.read_csv('toy_test_df.csv')

In [28]:
' '.join(toy_train['wav_filename'].str.rsplit('/',1, expand=True)[1].tolist())

'common_voice_sw_28704519.wav common_voice_sw_28704530.wav common_voice_sw_28704537.wav common_voice_sw_28704542.wav common_voice_sw_28704543.wav common_voice_sw_28704552.wav common_voice_sw_28704555.wav common_voice_sw_28704556.wav common_voice_sw_28704557.wav common_voice_sw_28704586.wav common_voice_sw_28704588.wav common_voice_sw_28704591.wav common_voice_sw_28704593.wav common_voice_sw_28704689.wav common_voice_sw_28704691.wav common_voice_sw_28704692.wav common_voice_sw_28704693.wav common_voice_sw_28704704.wav common_voice_sw_29790864.wav common_voice_sw_29790865.wav common_voice_sw_29790911.wav common_voice_sw_29790913.wav common_voice_sw_29790915.wav common_voice_sw_29790918.wav common_voice_sw_29790937.wav common_voice_sw_29790938.wav common_voice_sw_29790939.wav common_voice_sw_29790940.wav common_voice_sw_29790941.wav common_voice_sw_29790970.wav common_voice_sw_29790978.wav common_voice_sw_29790980.wav common_voice_sw_29790981.wav common_voice_sw_29790998.wav common_voice_

In [8]:
' '.join(toy_test['wav_filename'].str.rsplit('/',1, expand=True)[1].tolist())

'common_voice_sw_29002392.wav common_voice_sw_31292754.wav common_voice_sw_30612341.wav common_voice_sw_31291248.wav common_voice_sw_31295894.wav common_voice_sw_28581699.wav common_voice_sw_30667303.wav common_voice_sw_30342543.wav common_voice_sw_31229192.wav common_voice_sw_30644559.wav common_voice_sw_30621784.wav common_voice_sw_31384593.wav common_voice_sw_30644508.wav common_voice_sw_30644154.wav common_voice_sw_30644782.wav common_voice_sw_31414750.wav common_voice_sw_28266617.wav common_voice_sw_31291434.wav common_voice_sw_30577673.wav common_voice_sw_31081002.wav common_voice_sw_28261304.wav common_voice_sw_30562357.wav common_voice_sw_30563098.wav common_voice_sw_30642167.wav common_voice_sw_30644575.wav common_voice_sw_30562189.wav common_voice_sw_31306454.wav common_voice_sw_30620847.wav common_voice_sw_30644808.wav common_voice_sw_31292777.wav common_voice_sw_31152268.wav common_voice_sw_30386544.wav common_voice_sw_30623659.wav common_voice_sw_31255561.wav common_voice_

In [9]:
' '.join(toy_dev['wav_filename'].str.rsplit('/',1, expand=True)[1].tolist())

'common_voice_sw_28660554.wav common_voice_sw_28660555.wav common_voice_sw_28660556.wav common_voice_sw_28660557.wav common_voice_sw_28660558.wav common_voice_sw_28669831.wav common_voice_sw_28669832.wav common_voice_sw_28669833.wav common_voice_sw_28669835.wav common_voice_sw_28669836.wav common_voice_sw_28669837.wav common_voice_sw_28669838.wav common_voice_sw_28669839.wav common_voice_sw_28669840.wav common_voice_sw_28669841.wav common_voice_sw_28669842.wav common_voice_sw_28669843.wav common_voice_sw_28669844.wav common_voice_sw_28669845.wav common_voice_sw_28669846.wav common_voice_sw_28669847.wav common_voice_sw_28669848.wav common_voice_sw_28669849.wav common_voice_sw_28669876.wav common_voice_sw_28669877.wav common_voice_sw_28669878.wav common_voice_sw_28669879.wav common_voice_sw_28669880.wav common_voice_sw_28669901.wav common_voice_sw_28669903.wav common_voice_sw_28669904.wav common_voice_sw_28669922.wav common_voice_sw_28669923.wav common_voice_sw_28669924.wav common_voice_

In [10]:
toy_train['wav_filename'].str.rsplit('/',1, expand=True)[1]

0        common_voice_sw_28704519.wav
1        common_voice_sw_28704530.wav
2        common_voice_sw_28704537.wav
3        common_voice_sw_28704542.wav
4        common_voice_sw_28704543.wav
                     ...             
19856    common_voice_sw_30378060.wav
19857    common_voice_sw_30378843.wav
19858    common_voice_sw_30378857.wav
19859    common_voice_sw_30378968.wav
19860    common_voice_sw_30380015.wav
Name: 1, Length: 19861, dtype: object

In [11]:
toy_train['wav_filename'] = toy_train['wav_filename'].str.rsplit('/',1, expand=True)[1]
toy_dev['wav_filename'] = toy_dev['wav_filename'].str.rsplit('/',1, expand=True)[1]
toy_test['wav_filename'] = toy_test['wav_filename'].str.rsplit('/',1, expand=True)[1]

In [12]:
toy_train.to_csv('toy_train_df.csv', index=False)
toy_dev.to_csv('toy_dev_df.csv', index=False)
toy_test.to_csv('toy_test_df.csv', index=False)